In [101]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

import os
import cv2
import torch
import scipy
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torchvision.transforms.functional as TF
from PIL import Image
import matplotlib.pyplot as plt
from scipy import io as sio
from scipy import ndimage, misc
import csv
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [0]:
def initialize_model(num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    model_ft = models.alexnet(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
    input_size = 224

   
    return model_ft, input_size

  

In [0]:
class create_dataset(Dataset):
   
    def __init__(self,root_dir,total_count, csv_file, transform=None):
      
      
        self.root_dir = root_dir
        self.total_count = total_count
        self.csv_file = np.array(pd.read_csv(csv_file,header=None))[0]
        self.transform = transform

    def __len__(self):
        return self.total_count

    def __getitem__(self, image_no):
        img_name = os.path.join(self.root_dir,
                                str(image_no+1)+".jpg")
        image = Image.open(img_name)
        sample = {'image': image, 'label': self.csv_file[image_no]-1}

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

In [106]:

num_classes = 8

batch_size = 8

num_epochs = 15

feature_extract = True

alexnet = models.alexnet(pretrained=True)

alexnet.classifier[6] = nn.Linear(4096,8)

alexnet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

In [0]:
train_labels = np.array(pd.read_csv('/content/drive/My Drive/CV_assign3_dataset/train_labels.csv',header =None))[0]
test_labels = np.array(pd.read_csv('/content/drive/My Drive/CV_assign3_dataset/test_labels.csv',header =None))[0]

trainDataset = create_dataset('/content/drive/My Drive/CV_assign3_dataset/train',1888,'/content/drive/My Drive/CV_assign3_dataset/train_labels.csv',transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))

testDataset = create_dataset('/content/drive/My Drive/CV_assign3_dataset/test',800,'/content/drive/My Drive/CV_assign3_dataset/test_labels.csv',transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))

In [108]:

model_ft, input_size = initialize_model(8, feature_extract, use_pretrained=True)
print(model_ft)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

In [109]:


print("Initializing Datasets and Dataloaders...")


trainDataloader = torch.utils.data.DataLoader(trainDataset,batch_size = 8, shuffle = True, num_workers=4)
testDataloader = torch.utils.data.DataLoader(testDataset,batch_size = 8, shuffle = True, num_workers=4)
Dataloaders_dict = {trainDataloader,testDataloader}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [110]:

model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)


optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)


Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [111]:
criterion = nn.CrossEntropyLoss()

for epoch in range(15):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainDataloader, 0):
        # get the inputs
        inputs, labels = data['image'],data['label']
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer_ft.zero_grad()
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_ft.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Training done')

[1,    10] loss: 1.706
[1,    20] loss: 0.663
[1,    30] loss: 0.684
[1,    40] loss: 0.348
[1,    50] loss: 0.473
[1,    60] loss: 0.355
[1,    70] loss: 0.594
[1,    80] loss: 0.435
[1,    90] loss: 0.377
[1,   100] loss: 0.539
[1,   110] loss: 0.482
[1,   120] loss: 0.412
[1,   130] loss: 0.467
[1,   140] loss: 0.272
[1,   150] loss: 0.492
[1,   160] loss: 0.485
[1,   170] loss: 0.318
[1,   180] loss: 0.546
[1,   190] loss: 0.358
[1,   200] loss: 0.524
[1,   210] loss: 0.419
[1,   220] loss: 0.580
[1,   230] loss: 0.520
[2,    10] loss: 0.362
[2,    20] loss: 0.421
[2,    30] loss: 0.232
[2,    40] loss: 0.471
[2,    50] loss: 0.688
[2,    60] loss: 0.327
[2,    70] loss: 0.596
[2,    80] loss: 0.443
[2,    90] loss: 0.437
[2,   100] loss: 0.441
[2,   110] loss: 0.533
[2,   120] loss: 0.377
[2,   130] loss: 0.330
[2,   140] loss: 0.649
[2,   150] loss: 0.363
[2,   160] loss: 0.312
[2,   170] loss: 0.266
[2,   180] loss: 0.251
[2,   190] loss: 0.403
[2,   200] loss: 0.294
[2,   210] 

In [112]:
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(testDataloader, 0):
        # get the inputs
        inputs, labels = data['image'],data['label']
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model_ft(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy of the network on the 800 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 800 test images: 90 %
